In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# For example, here's several helpful packages to load



import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



# Input data files are available in the read-only "../input/" directory

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



import os

# for dirname, _, filenames in os.walk('/kaggle/input'):

#     for filename in filenames:

#         print(os.path.join(dirname, filename))



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be s

In [2]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

In [3]:
import os

import torch

import pandas as pd

import numpy as np

from torch.utils.data import Dataset, random_split, DataLoader

from PIL import Image

import torchvision.models as models

from tqdm.notebook import tqdm

import torchvision.transforms as T

import torch.nn.functional as F

import torch.nn as nn

from torchvision.utils import make_grid



import os

import re

import requests



import matplotlib.pyplot as plt

%matplotlib inline



import warnings

warnings.filterwarnings('ignore')

In [4]:
model_category = 'Women Tops & Tunics'

In [ ]:
train_df_Men_Tshirts = pd.read_csv('../../Preprocessor-FillNA/output/train_fillna_Women Tops Tunics_effnet_b5_4epochs.csv')

# train_df_Men_Tshirts = train_df_Men_Tshirts.head(100)

In [6]:
train_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
0,51375,Women Tops & Tunics,10,black,regular,crop,round neck,casual,default,default,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/051...
1,51376,Women Tops & Tunics,10,navy blue,fitted,crop,high,casual,default,solid,short sleeves,default,knitted,/kaggle/input/visual-taxonomy/train_images/051...
2,51377,Women Tops & Tunics,10,red,regular,regular,round neck,casual,printed,typography,sleeveless,sleeveless,knitted,/kaggle/input/visual-taxonomy/train_images/051...
3,51378,Women Tops & Tunics,10,default,fitted,crop,stylised,casual,solid,solid,short sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/051...
4,51379,Women Tops & Tunics,10,default,boxy,regular,round neck,casual,printed,typography,short sleeves,default,tie-ups,/kaggle/input/visual-taxonomy/train_images/051...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18999,70374,Women Tops & Tunics,10,multicolor,fitted,regular,square neck,casual,printed,default,short sleeves,regular sleeves,ruffles,/kaggle/input/visual-taxonomy/train_images/070...
19000,70375,Women Tops & Tunics,10,yellow,regular,crop,round neck,casual,default,default,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/070...
19001,70376,Women Tops & Tunics,10,maroon,fitted,crop,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/070...
19002,70377,Women Tops & Tunics,10,green,regular,regular,high,party,solid,solid,short sleeves,default,ruffles,/kaggle/input/visual-taxonomy/train_images/070...


In [7]:
attr_columns = train_df_Men_Tshirts.filter(like='attr_').columns.to_list() # Adjust if more attributes exist

print(attr_columns)

['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']


In [8]:
from sklearn.preprocessing import LabelEncoder



# Initialize LabelEncoder for each attribute column

label_encoders = {}

for column in attr_columns:

    le = LabelEncoder()

    train_df_Men_Tshirts[column] = le.fit_transform(train_df_Men_Tshirts[column])

    label_encoders[column] = le  # Store the encoder for inverse transformation later if needed



# Check the updated DataFrame

# train_df_Men_Tshirts = train_df_Men_Tshirts.head()

In [9]:
from sklearn.model_selection import train_test_split



# Split the dataset into training and validation sets

train_df1, val_df1 = train_test_split(train_df_Men_Tshirts, test_size=0.15, random_state=42)

In [10]:
train_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
12525,63900,Women Tops & Tunics,10,8,3,1,2,0,0,0,1,2,2,/kaggle/input/visual-taxonomy/train_images/063...
8406,59781,Women Tops & Tunics,10,5,3,1,2,0,0,0,1,2,3,/kaggle/input/visual-taxonomy/train_images/059...
17346,68721,Women Tops & Tunics,10,6,2,0,6,0,2,0,0,3,2,/kaggle/input/visual-taxonomy/train_images/068...
7588,58963,Women Tops & Tunics,10,8,2,0,1,0,2,4,2,3,2,/kaggle/input/visual-taxonomy/train_images/058...
17772,69147,Women Tops & Tunics,10,2,2,1,5,0,2,4,1,1,2,/kaggle/input/visual-taxonomy/train_images/069...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,62659,Women Tops & Tunics,10,2,3,1,6,0,2,4,1,1,2,/kaggle/input/visual-taxonomy/train_images/062...
11964,63339,Women Tops & Tunics,10,10,2,0,2,0,1,3,1,2,4,/kaggle/input/visual-taxonomy/train_images/063...
5390,56765,Women Tops & Tunics,10,1,3,1,6,0,2,4,1,0,2,/kaggle/input/visual-taxonomy/train_images/056...
860,52235,Women Tops & Tunics,10,2,1,1,0,0,0,1,3,2,5,/kaggle/input/visual-taxonomy/train_images/052...


In [11]:
val_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
1911,53286,Women Tops & Tunics,10,4,2,1,1,0,2,4,1,2,2,/kaggle/input/visual-taxonomy/train_images/053...
13144,64519,Women Tops & Tunics,10,8,3,1,1,0,1,1,3,0,2,/kaggle/input/visual-taxonomy/train_images/064...
2929,54304,Women Tops & Tunics,10,1,3,1,2,0,2,4,0,2,2,/kaggle/input/visual-taxonomy/train_images/054...
2968,54343,Women Tops & Tunics,10,2,3,1,1,0,2,4,1,2,0,/kaggle/input/visual-taxonomy/train_images/054...
10407,61782,Women Tops & Tunics,10,3,3,0,2,0,2,4,0,2,2,/kaggle/input/visual-taxonomy/train_images/061...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18011,69386,Women Tops & Tunics,10,8,1,1,0,1,2,4,2,0,1,/kaggle/input/visual-taxonomy/train_images/069...
6844,58219,Women Tops & Tunics,10,8,2,1,3,0,2,4,1,2,5,/kaggle/input/visual-taxonomy/train_images/058...
13951,65326,Women Tops & Tunics,10,10,0,0,2,0,1,5,1,2,1,/kaggle/input/visual-taxonomy/train_images/065...
9326,60701,Women Tops & Tunics,10,11,3,0,2,0,1,5,2,3,1,/kaggle/input/visual-taxonomy/train_images/060...


In [12]:
import torch

import torch.nn as nn

import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

from torchvision import models, transforms

from PIL import Image

import pandas as pd

import numpy as np



# Define custom dataset

class CustomDataset(Dataset):

    def __init__(self, dataframe, img_dir, transform=None, is_test=False):

        self.dataframe = dataframe

        self.img_dir = img_dir

        self.transform = transform

        self.is_test = is_test  # Flag to indicate if it's test set without labels



    def __len__(self):

        return len(self.dataframe)



    def __getitem__(self, idx):

        img_path = self.img_dir + self.dataframe.iloc[idx]['image_path']

        image = Image.open(img_path).convert('RGB')

        

        if self.transform:

            image = self.transform(image)

        

        if self.is_test:  # For test set, just return the image without labels

            return image

        

        # For train/validation set, return image and labels

        labels = self.dataframe.iloc[idx][attr_columns].values

        labels = labels.astype(np.int64)  # Ensure labels are integers

        labels = torch.tensor(labels, dtype=torch.long)

        

        return image, labels

    

    

# Transform

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Apply Color Jitter
    transforms.RandomHorizontalFlip(),  # Apply Horizontal Flip with 50% probability
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

    

train_dataset = CustomDataset(dataframe=train_df1, img_dir='', transform=transform)

val_dataset = CustomDataset(dataframe=val_df1, img_dir='', transform=transform)



train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [13]:
import ssl



ssl._create_default_https_context = ssl._create_stdlib_context

In [14]:
from torchvision import models

available_models = dir(models)



# Filter out private members and display model names

available_models = [model for model in available_models if not model.startswith('_')]

print("Available models in torchvision.models:")

print(available_models)

Available models in torchvision.models:
['AlexNet', 'AlexNet_Weights', 'ConvNeXt', 'ConvNeXt_Base_Weights', 'ConvNeXt_Large_Weights', 'ConvNeXt_Small_Weights', 'ConvNeXt_Tiny_Weights', 'DenseNet', 'DenseNet121_Weights', 'DenseNet161_Weights', 'DenseNet169_Weights', 'DenseNet201_Weights', 'EfficientNet', 'EfficientNet_B0_Weights', 'EfficientNet_B1_Weights', 'EfficientNet_B2_Weights', 'EfficientNet_B3_Weights', 'EfficientNet_B4_Weights', 'EfficientNet_B5_Weights', 'EfficientNet_B6_Weights', 'EfficientNet_B7_Weights', 'EfficientNet_V2_L_Weights', 'EfficientNet_V2_M_Weights', 'EfficientNet_V2_S_Weights', 'GoogLeNet', 'GoogLeNetOutputs', 'GoogLeNet_Weights', 'Inception3', 'InceptionOutputs', 'Inception_V3_Weights', 'MNASNet', 'MNASNet0_5_Weights', 'MNASNet0_75_Weights', 'MNASNet1_0_Weights', 'MNASNet1_3_Weights', 'MaxVit', 'MaxVit_T_Weights', 'MobileNetV2', 'MobileNetV3', 'MobileNet_V2_Weights', 'MobileNet_V3_Large_Weights', 'MobileNet_V3_Small_Weights', 'RegNet', 'RegNet_X_16GF_Weights', '

In [15]:
import torch

import torch.nn as nn

import torch.optim as optim

from torchvision import models



class MultiOutputModel1(nn.Module):

    def __init__(self, num_classes):

        super(MultiOutputModel1, self).__init__()

        # Use MobileNetV2 as the base model

        self.base_model = models.mobilenet_v3_large(pretrained=True)

        self.base_model.classifier = nn.Identity()  # Remove original classification layer

        

        # Add an adaptive pooling layer to make sure output is flat

        self.pooling = nn.AdaptiveAvgPool2d((1, 1))  # Convert 2D output to 1D

        

        # Dynamically create a fully connected layer for each attribute

        self.output_layers = nn.ModuleDict()

        for attr, n_classes in num_classes.items():

            self.output_layers[attr] = nn.Linear(960, n_classes)  # Adjust input to 1280 for MobileNetV2



    def forward(self, x):

        x = self.base_model.features(x)  # Extract features

        x = self.pooling(x)  # Adaptive pool to (1, 1) shape

        x = torch.flatten(x, 1)  # Flatten the output to (batch_size, 1280)

        

        outputs = {}

        # Dynamically compute output for each attribute

        for attr, layer in self.output_layers.items():

            outputs[attr] = layer(x)

        

        return list(outputs.values())



# Example usage

num_classes = {}

for key in attr_columns:

    num_classes[key] = train_df_Men_Tshirts[key].nunique()



model = MultiOutputModel1(num_classes)



# Loss and optimizer

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 142MB/s] 


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)



# Training loop

def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path=f"../../Models/Women Tops & Tunics/mobile_net_v3_large_fillna_eff_net_{model_category}.pth"):

    scheduler = optim.lr_scheduler.StepLR(optimizer = optimizer , step_size=5, gamma=0.5)

    best_val_loss = float('inf')

    for epoch in tqdm(range(num_epochs)):

        model.train()

        train_loss = 0.0

        for images, labels in train_loader:

            images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device

            optimizer.zero_grad()

            outputs = model(images)

            loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])

            loss.backward()

            optimizer.step()

            train_loss += loss.item()

        scheduler.step()

        

        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}')



        # Validation

        model.eval()

        val_loss = 0.0

        with torch.no_grad():

            for images, labels in val_loader:

                images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device

                outputs = model(images)

                loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])

                val_loss += loss.item()

        

        avg_val_loss = val_loss / len(val_loader)

        print(f'Epoch {epoch+1}, Val Loss: {val_loss/len(val_loader)}')

        

        if avg_val_loss < best_val_loss:

            best_val_loss = avg_val_loss

            torch.save(model.state_dict(), save_path)

            print(f"Model saved with Val Loss: {best_val_loss:.4f} at Epoch {epoch+1}")



# Run training

train(model, train_loader, val_loader, criterion, optimizer, num_epochs=9)



model.load_state_dict(torch.load(f"../../Models/Women Tops & Tunics/mobile_net_v3_large_fillna_eff_net_{model_category}.pth"))

  0%|          | 0/9 [00:00<?, ?it/s]

Epoch 1, Train Loss: 6.648301176741572
Epoch 1, Val Loss: 4.46011155810436
Model saved with Val Loss: 4.4601 at Epoch 1
Epoch 2, Train Loss: 4.250604194933826
Epoch 2, Val Loss: 3.7113162878505346
Model saved with Val Loss: 3.7113 at Epoch 2
Epoch 3, Train Loss: 3.5973206140027187
Epoch 3, Val Loss: 3.333046530212104
Model saved with Val Loss: 3.3330 at Epoch 3
Epoch 4, Train Loss: 3.1756484316127134
Epoch 4, Val Loss: 3.187316317798039
Model saved with Val Loss: 3.1873 at Epoch 4
Epoch 5, Train Loss: 2.9024380282600326
Epoch 5, Val Loss: 3.080351376666703
Model saved with Val Loss: 3.0804 at Epoch 5
Epoch 6, Train Loss: 2.556598657664686
Epoch 6, Val Loss: 2.9650129166395303
Model saved with Val Loss: 2.9650 at Epoch 6
Epoch 7, Train Loss: 2.397083756947281
Epoch 7, Val Loss: 2.9564506861084667
Model saved with Val Loss: 2.9565 at Epoch 7
Epoch 8, Train Loss: 2.2993774182135516
Epoch 8, Val Loss: 2.967812188510788
Epoch 9, Train Loss: 2.1896855954486543
Epoch 9, Val Loss: 2.9731833368

<All keys matched successfully>